In [3]:
import random
import numpy as np
from tqdm import tqdm
import time
from functools import partial
import scipy as sp
import joblib
import gc
from sklearn.model_selection import StratifiedKFold
import re
import math
import json
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

from layers.losses import LabelSmoothingCrossEntropy, FocalLoss, LabelSM_Focal, DiceLoss, LabelSmoothingCrossEntropyWeight
from torch.nn import CrossEntropyLoss
from layers.adversarial import FGM, PGD, Lookahead

from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from sklearn.metrics import accuracy_score

from NEZHA.modeling_nezha import NeZhaModel
from NEZHA.configuration_nezha import NeZhaConfig
# from transformers import NezhaModel
# from transformers import NezhaConfig
from models import BertLastFourEmbeddingsPooler, BertLastTwoCls, BertLastFourCls

from transformers import *
import logging
print(torch.__version__)
import warnings
import transformers
transformers.logging.set_verbosity_error()
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split

1.10.1+cu111
1.10.1+cu111


In [ ]:
# [[1, 0, 0],
#  [0, 1, 0]]

# [[L00, L10, L20],
#  [L01, L11, L21]]

# [[L00, LO1],
#  [L10, L11],
#  [L20, L21]]

# --> []
# [2, 3] * [3, 2]
# [2, 2]

In [2]:
class args(object):
    
    def __init__(self):
        # # bert_base_UER chinese_macbert_base chinese_roberta_wwm_ext bert_large_UER chinese_macbert_large roberta-36
#         self.model_checkpoint =  '/root/autodl-tmp/pretrained_models/NEZHA-base-wwm'
        
        self.model_checkpoint =  '/root/autodl-tmp/CCF-小样本/Nezha_pytorch/nezha_model'
        self.model_type = 'nezha'
#         self.model_checkpoint =  '/root/autodl-tmp/CCF-小样本/Nezha_pytorch/pretrain/outputs'
        self.device = 'cuda'
        self.max_length = 412
        self.bert_dim = 768
        
        self.num_reinit_layers = 1
        self.reinit_pooler = False
        self.layerwise_learning_rate_decay = 1
        
        self.bert_dim = 768
        
args = args()
tokenizer = BertTokenizer.from_pretrained(args.model_checkpoint)

In [3]:
    
with open('./data/testB.json', 'r')as f:
    test_data = f.readlines()
    test_data = [eval(i.strip())for i in test_data]

In [4]:
test_data_all = []
for idx, i in enumerate(test_data):
    i['data_idx']=idx
    test_data_all.append(i)
len(test_data_all)

20890

In [5]:
max_len = args.max_length - 4

test_dict ={'input_ids': [], 'token_type_ids': [], 'attention_mask': [], 'input_lengths':[], 'data_idx': []}

for idx, test_data_ in tqdm(enumerate(test_data_all)):
    title = test_data_['title']
    assignee = test_data_['assignee']
    abstract = test_data_['abstract']
    
    title_ids_ori = tokenizer.encode(title, add_special_tokens=False)
    assignee_ids_ori = tokenizer.encode(assignee, add_special_tokens=False)
    abstract_ids_ori = tokenizer.encode(abstract, add_special_tokens=False)
    
    title_len = len(title_ids_ori)
    assignee_len = len(assignee_ids_ori)
    abstract_len = len(abstract_ids_ori)
    
    abstract_cutlen = max_len - title_len - assignee_len
    
    if abstract_cutlen >= abstract_len:
        token_ids = [101] + title_ids_ori + [102] + assignee_ids_ori + [102] + abstract_ids_ori + [102]
    else:
        mid_len = int(abstract_cutlen / 2)
        token_ids = [101] + title_ids_ori + [102] + assignee_ids_ori + [102] + abstract_ids_ori[: mid_len] + abstract_ids_ori[-mid_len: ] + [102]
    token_type_ids = [0] * len(token_ids)
    attention_mask = [1] * len(token_ids)
    input_lengths = len(token_ids)
    
    test_dict['input_ids'].append(token_ids)
    test_dict['token_type_ids'].append(token_type_ids)
    test_dict['attention_mask'].append(attention_mask)
    test_dict['input_lengths'].append(input_lengths)
    test_dict['data_idx'].append(idx)

        
test_dict = {k: np.array(test_dict[k]) for k in test_dict}
# In[6]:

20890it [01:01, 341.56it/s]


In [6]:
class WBDataset(Dataset):

    def __init__(self, data, tokenizer, batch_first=True, test=False):
        self.data = data
        self.tokenizer = tokenizer
        self.pad = tokenizer.pad_token_id
        self.batch_first = batch_first
        self.test = test

    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, index):
        instance = {}
        instance['input_ids'] = self.data['input_ids'][index]
        instance['token_type_ids'] = self.data['token_type_ids'][index]
        instance['attention_mask'] = self.data['attention_mask'][index]
        instance['input_lengths'] = self.data['input_lengths'][index]
        if not self.test:
            instance['labels'] = self.data['labels'][index]
        else:
            instance['data_idx'] = self.data['data_idx'][index]
        
        return instance

    def collate(self, batch):
        
        input_ids = pad_sequence(
            [torch.tensor(instance["input_ids"], dtype=torch.long) for instance in batch],
            batch_first=self.batch_first, padding_value=self.pad)
        
        token_type_ids = pad_sequence(
            [torch.tensor(instance["token_type_ids"], dtype=torch.long) for instance in batch],
            batch_first=self.batch_first, padding_value=self.pad)
        
        attention_mask = pad_sequence(
            [torch.tensor(instance["attention_mask"], dtype=torch.long) for instance in batch],
            batch_first=self.batch_first, padding_value=self.pad)
        
        input_lengths = torch.tensor([torch.tensor(instance["input_lengths"], dtype=torch.int) for instance in batch])
        
        if not self.test:
            labels = torch.tensor([torch.tensor(instance["labels"], dtype=torch.long) for instance in batch])

            return input_ids, token_type_ids, attention_mask, input_lengths, labels
        else:
            data_idx = torch.tensor([torch.tensor(instance["data_idx"], dtype=torch.long) for instance in batch])
            
            return input_ids, token_type_ids, attention_mask, input_lengths, data_idx
        
test_batch_size = 64
test_dataset =  WBDataset(test_dict, tokenizer, test=True)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, collate_fn=test_dataset.collate, shuffle=False, num_workers=4)

In [7]:

def test_model(model, test_loader, test_count):
    
    test_preds_fold = np.zeros((test_count, 36)) ####
    
    model.eval()
    tk0 = tqdm(test_loader, total=len(test_loader))
    with torch.no_grad():
        for i, batch in enumerate(tk0):
            batch = tuple(t.to(device) for t in batch)
            ids, segids, mask, lens, y_truth = batch

            y_pred = model(
                input_ids = ids, 
                input_mask = mask, 
                input_segids = segids,
                input_lengths = lens,
            )[0] 

            y_pred = torch.softmax(y_pred, dim=-1).cpu().detach().numpy()
            test_bidx= i * test_batch_size
            test_eidx= (i + 1) * test_batch_size
            test_preds_fold[test_bidx:test_eidx] = y_pred
            
    return test_preds_fold

In [12]:
import os
file_names = []
file_path = r'./best_models/b_pseudo/'
#提取文件中的所有文件生成一个列表
folders = os.listdir(file_path)
for file in folders:
    if '.bin' in file:
        file_names.append(file_path + file)
        file_names.sort()

In [14]:
b_test_preds = {}


for file_name in file_names[:]:
    if 'LastFourCls' in file_name:
        model = modelBertLastFourCls
    elif 'LastFourEmbeddingsPooler' in file_name:
        model = modelBertLastFourEmbeddingsPooler
    elif 'LastTwoCls' in file_name:
        model = modelBertLastTwoCls
    else:
        print(fine_name)
    
    model.load_state_dict(torch.load(file_name))
    test_pred = test_model(model, test_loader, len(test_data_all))
    b_test_preds[file_name] = test_pred

100%|██████████| 327/327 [01:47<00:00,  3.04it/s]


In [15]:
import os
file_names = []
file_path = r'./best_models/curbert_6415_1103/'
#提取文件中的所有文件生成一个列表
folders = os.listdir(file_path)
for file in folders:
    if '.bin' in file:
        file_names.append(file_path + file)
        file_names.sort()
file_names

['./best_models/curbert_6415_1103/nazhe_base-pseudo_labels_6410-BertLastFourEmbeddingsPooler-ls-fgm-seed2020-gkf2020-layerwise1-wd0.01-warmp0.-n_splits5-grad_acc1-num_epochs10-3e-05-32_fold_0.bin',
 './best_models/curbert_6415_1103/nazhe_base-pseudo_labels_6410-BertLastFourEmbeddingsPooler-ls-fgm-seed2020-gkf2020-layerwise1-wd0.01-warmp0.-n_splits5-grad_acc1-num_epochs10-3e-05-32_fold_1.bin',
 './best_models/curbert_6415_1103/nazhe_base-pseudo_labels_6410-BertLastFourEmbeddingsPooler-ls-fgm-seed2020-gkf2020-layerwise1-wd0.01-warmp0.-n_splits5-grad_acc1-num_epochs10-3e-05-32_fold_2.bin',
 './best_models/curbert_6415_1103/nazhe_base-pseudo_labels_6410-BertLastFourEmbeddingsPooler-ls-fgm-seed2020-gkf2020-layerwise1-wd0.01-warmp0.-n_splits5-grad_acc1-num_epochs10-3e-05-32_fold_3.bin',
 './best_models/curbert_6415_1103/nazhe_base-pseudo_labels_6410-BertLastFourEmbeddingsPooler-ls-fgm-seed2020-gkf2020-layerwise1-wd0.01-warmp0.-n_splits5-grad_acc1-num_epochs10-3e-05-32_fold_4.bin']

In [16]:
for file_name in file_names[:]:
    if 'LastFourCls' in file_name:
        model = modelBertLastFourCls
    elif 'LastFourEmbeddingsPooler' in file_name:
        model = modelBertLastFourEmbeddingsPooler
    elif 'LastTwoCls' in file_name:
        model = modelBertLastTwoCls
    else:
        print(fine_name)
    
    model.load_state_dict(torch.load(file_name))
    test_pred = test_model(model, test_loader, len(test_data_all))
    b_test_preds[file_name] = test_pred
    
    

100%|██████████| 327/327 [01:47<00:00,  3.05it/s]


In [17]:
len(b_test_preds)

15

In [18]:
with open('./data/results_dict.pk', 'wb') as f:
    joblib.dump(b_test_preds, f)

In [9]:
device = torch.device('cuda')
modelBertLastFourEmbeddingsPooler = BertLastFourEmbeddingsPooler(args).to(device)
modelBertLastTwoCls = BertLastTwoCls(args).to(device)
modelBertLastFourCls = BertLastFourCls(args).to(device)

In [10]:
test_preds = {}


for file_name in file_names[:]:
    if 'LastFourCls' in file_name:
        model = modelBertLastFourCls
    elif 'LastFourEmbeddingsPooler' in file_name:
        model = modelBertLastFourEmbeddingsPooler
    elif 'LastTwoCls' in file_name:
        model = modelBertLastTwoCls
    else:
        print(fine_name)
    
    model.load_state_dict(torch.load(file_name))
    test_pred = test_model(model, test_loader, len(test_data_all))
    test_preds[file_name] = test_pred
    
#     test_preds.append(test_pred)

100%|██████████| 327/327 [01:46<00:00,  3.07it/s]


In [11]:
with open('./data/B_results_dict.pk', 'wb') as f:
    joblib.dump(test_preds, f)

In [39]:
df_pred = pd.read_csv('./data/submit_example_B.csv')

final_preds = np.zeros(test_preds[0].shape)
for idx in range(len(test_preds)):
    test_preds_fold = test_preds[idx]
    final_preds += test_preds_fold  / len(test_preds)
    
final_reses = np.argmax(final_preds, axis=1)

final_sub = []
final_sub_ = []
for i in range(len(df_pred)):
    final_res = final_reses[i]
    final_sub_.append(str(final_res))
    
    
final_reses_vote = {}
for idx in range(len(test_preds)):
    test_preds_fold = test_preds[idx]
    
    for i in range(test_preds_fold.shape[0]):
        if i not in final_reses_vote:
            final_reses_vote[i] = {i:0 for i in range(36)}
        sample_res = np.argmax(test_preds_fold[i], axis=0)
        final_reses_vote[i][sample_res] += 1
 

final_sub_vote_ = []
for idx, sample_id in enumerate(final_reses_vote):
    
    
    sample_label = -1
    sample_label_count = final_reses_vote[sample_id]
    sample_label_sort =  sorted(sample_label_count.items(), key=lambda d:d[1], reverse=True)
    
    if sample_label_sort[0][1] > sample_label_sort[1][1]:
        sample_label = str(sample_label_sort[0][0])
    else:
        sample_label = np.argmax(final_preds[sample_id], axis=0)
        
    final_sub_vote_.append(str(sample_label))

In [93]:
# pseudo label_id

pseudo_data = test_data.copy()
for label, pseudo_data_ in zip(final_sub_vote_, pseudo_data):
    pseudo_data_['label_id'] = label
    
with open('./data/pseudo_B_1108_v1.json', 'w') as f:
    json.dump(pseudo_data, f)

In [94]:
pseudo_data[:5]

[{'id': '487fdc38d8b39d35ba18206b835e57e3',
  'title': '建立灵活以太网路径的方法和网络设备',
  'assignee': '华为技术有限公司',
  'abstract': '本申请提供了一种建立FlexE路径的方法和网络设备，能够降低节点的控制面的管理复杂度和信令开销。该方法包括：第一转发节点接收第一源节点发送的第一路径建立请求消息，第一路径建立请求消息用于建立第一源节点到第一目的节点之间的第一FlexE路径，第一FlexE路径用于传输第一业务流；第一转发节点根据第一路径建立请求消息，建立第一转发节点至第二转发节点之间的第二FlexE路径，并在建立第二FlexE路径后，删除已建立的第一转发节点到第二转发节点之间的第三FlexE路径；第一转发节点建立并保存第一FlexE路径和第二FlexE路径的对应关系，使得将第一业务流和第二业务流映射至第二FlexE路径的时隙上。',
  'data_idx': 0,
  'label_id': '0'},
 {'id': '32e267e4a7bc6bd000fcd45616e448d7',
  'title': '一种程序代码标记方法及装置',
  'assignee': '阿里巴巴集团控股有限公司',
  'abstract': '本申请公开了一种程序代码标记方法及装置，该方法包括：顺序读取程序代码，当读取到程序代码中包含的切换标识符时，确定该切换标识符对应的标记规则，并采用该标记规则，对后续读取到的程序代码进行标记，直到再次读取到程序代码中包含的下一个切换标识符时，再次确定该下一个切换标识符对应的另一标记规则，并采用该另一标记规则，继续对读取到的程序代码进行标记，直到将全部的程序代码标记完为止。通过上述方法，即使程序代码由两种以上的编程语言进行编写的，但是可以根据切换标识符确定需要使用哪个标记规则，对后续读取到的程序代码进行准确的标记。',
  'data_idx': 1,
  'label_id': '4'},
 {'id': '9b808e8060a5e84886e4fb1bf94699d4',
  'title': '一种新型环境保护净化器',
  'assignee': '李菊红',
  'abstract': '本实用新型适用于环境保护技术领域，提供了一种

In [84]:
with open('./data/pseudo_labels_6410.json', 'r') as f:
    pseudo_data_a = f.readlines()
    pseudo_data_a = [eval(i.strip())for i in pseudo_data_a]
    
label_a = [i['label_id'] for i in pseudo_data_a]

In [78]:
def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
seed_all(20201227)

In [60]:
array = np.array(list(range(len(test_data))))
labels = final_sub_vote_

In [89]:
array_ = np.array(list(range(len(test_data))))
labels = final_sub_vote_

pseudo_idxes_B = []
for i in range(20):
    train_idxes, test_idxes, _, _ = train_test_split(np.expand_dims(array,axis=-1), labels, train_size=0.3, stratify=labels, random_state = 666)
    train_idxes = train_idxes.squeeze()
    pseudo_idxes_B.append(train_idxes)
    
    

array_a = np.array(list(range(len(pseudo_data_a))))
labels_a = label_a   
    
pseudo_idxes_A = []
for i in range(20):
    train_idxes, test_idxes, _, _ = train_test_split(np.expand_dims(array_a,axis=-1), labels_a, train_size=0.3, stratify=labels_a, random_state = 666)
    train_idxes = train_idxes.squeeze()
    pseudo_idxes_A.append(train_idxes)

In [96]:
# stratified_sample = train_test_split(np.expand_dims(array,axis=-1), labels, train_size=0.3, stratify=labels, random_state = 666)

with open('./data/pseudo_idxes.pk', 'wb') as f:
    joblib.dump({'pseudo_idxes_A': pseudo_idxes_A, 'pseudo_idxes_B': pseudo_idxes_B}, f)

In [82]:
train_idxes.squeeze()

array([ 8462, 10830, 14703, ...,  6739,  8841,  8855])

In [63]:
np.expand_dims(array,axis=0)

array([[    0,     1,     2, ..., 20887, 20888, 20889]])